# Segmenting and Clustering Neighborhoods in Toronto
In this assignment, I will be exploring the neighborhoods of a particular borough(district) in Toronto, Canada; North York.

I begin by extracting a list of all districts and neighborhoods in Toronto via a wikipage, and consolidating the neighborhoods by their postal codes into a dataframe. 

The exploration begins with probing the most common venue categories (such as cafes, supermarket etc) for one neighborhood in North York, using FourSquares APIs. This is then extended to all neighborhoods in North York within a 1000m radius.

Thereafter, I will be using K-means clustering to segment the neighborhoods into clusters ,using  the feature of most common venue for each neighborhood. This segmentation and clustering will be presented both visually with a map, and a tabular frame.

#### Importing libraries
Begin by importing all necessary libraries and packages required for data manipulation, clustering and visualization

In [1]:
#libraries for downloading and parsing webpages  
from bs4 import BeautifulSoup
import requests

#Libraries for structuring data for manipulation and analysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import csv

#Library for handling json files
import json

#Library for converting addresses into latitude and longitude
import geocoder
from geopy.geocoders import Nominatim

#Library for performing vectorized operations on data
import numpy as np

#Library for plotting visuals displays and maps
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

#Library for handling clustering algorithm
from sklearn.cluster import KMeans

print('All Libraries Imported')

All Libraries Imported


##  1. Data Retrieval and Extraction

Get the wikipage containing the list of  postal codes and their corresponding boroughs and neighborhoods in Toronto

In [2]:
#Retrieving the wikipage on list of postal codes in Toronto
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
page.status_code #A code of 200 indicates success

200

In [3]:
#Creating a BS4 object, for parsing webpage content
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
#Extracting the table containing postal codes, borough and neighborhood data
table = soup.find(class_='wikitable sortable')

In [7]:
#Writing each row in the table to a csv file
tablerows = table.find_all('tr')
f = csv.writer(open('toronto.csv', 'w'))
f.writerow(['PostalCode', 'Borough', 'Neighborhood'])

for a_row in tablerows:
    rdata = list(a_row.find_all('td'))
    if rdata == []:
        continue
    elif (rdata[1].get_text() =="Not assigned"):
        continue
    else:
        if (rdata[2].get_text() == 'Not assigned\n'):
            pcode = rdata[0].get_text('\r', strip = True)
            borough = rdata[1].get_text('\r', strip = True)
            nhood = rdata[1].get_text('\r', strip = True)
        else:
            pcode = rdata[0].get_text('\r', strip = True)
            borough = rdata[1].get_text('\r', strip = True)
            nhood = rdata[2].get_text('\r', strip = True)
     
        f.writerow([pcode, borough, nhood])

###### Transform the data into a pandas dataframe
A few intermediate dataframes will be created from different sources. 
But a final dataframe (toronto_df), containing all consolidated neighborhoods with their respective geographical coordinates will be used for exploration, segmentationa and clustering.

In [8]:
#Creating a dataframe for data analysis and manipulation
df = pd.read_csv('toronto.csv')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
#Creating a dataframe with the required column headers
toronto_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

#Inserting the first record into the dataframe
toronto_df = toronto_df.append(df.iloc[0,:], ignore_index = False, sort = False)
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods


In [10]:
#Consolidating neighborhoods that shares the same postal code, into one record 
toronto_index = 0

for index,row in df.iloc[1:].iterrows():
    if row['PostalCode'] == toronto_df.iloc[toronto_index,0]:
        toronto_df.iloc[toronto_index,2] = ",".join((toronto_df.iloc[toronto_index,2], row['Neighborhood']))
    
    else:
        new_row = pd.Series(row)
        toronto_df = toronto_df.append(new_row, ignore_index = True)
        toronto_index = toronto_index + 1

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
#Reading the csv file with geographical coordinates of the neighborhoods
lat_lng_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#Creating two new columns for each neighborhood's coordinates to the dataframe, and initializing with zero values
toronto_df['Latitude'] = 0.0
toronto_df['Longitude'] = 0.0
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",0.0,0.0
3,M6A,North York,"Lawrence Heights,Lawrence Manor",0.0,0.0
4,M7A,Queen's Park,Queen's Park,0.0,0.0


In [13]:
#Populating each neighborhood's coordinates with latitude and longitude data
for index1, row1 in toronto_df.iterrows():
    
    for index2, row2 in lat_lng_df.iterrows():
        
        if row1['PostalCode'] == row2['Postal Code']:
            toronto_df.iloc[index1,3] = lat_lng_df.iloc[index2,1]
            toronto_df.iloc[index1,4] = lat_lng_df.iloc[index2,2]
            break

toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


We summarize the number of boroughs and neighborhoods in Toronto. A neighborhood is defined as a community that has a distinct postal code. There communities that share the same postal code are grouped together as a neighborhood.

In [14]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(len(toronto_df['Borough'].unique()),toronto_df.shape[0]))

Toronto has 11 boroughs and 103 neighborhoods.


###### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
#Retrieving the geographical coordinates of Toronto
address = 'City of Toronto, TR'
geolocator = Nominatim(user_agent = 'tr_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of the City of Toronto are 43.6561136, -79.392321.


Creates a map of the City of Toronto using latitude and longitude coordinates, with markers indicating neighborhoods

In [16]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)


map_toronto

## 2. Exploring North York
Now I will zoom in to a specific borough, and explore it in greater details, such as extracting the category and number of each venue in this borough. With this feature information, I will attempt to segment and cluster the neighborhoods in this borough, based on this feature property.

I have selected North York as the borough of interest to explore. 

In [17]:
north_york = toronto_df[toronto_df['Borough'] == 'North York'].reset_index(drop = True)
north_york.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [18]:
# Retrieving the geographical coordinates of North York, for plotting map
address = 'North York, TR'

geolocator = Nominatim(user_agent = 'tr_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of North York are {}, {}'.format(latitude, longitude))

The geographical coordinate of North York are 43.7708175, -79.4132998


In [19]:
#Create map of North York, Toronto, using latitude and longitude values
map_north_york = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat,lng, label in zip(north_york['Latitude'], north_york['Longitude'], north_york['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius =5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_north_york)
        
map_north_york

###### Foursquare credentials
To be used for accessing Foursquare's APIs

In [20]:
CLIENT_ID = 'AVT01CGGJTLVXTNFWSABFLKPYKCRPHLAKFWBICJ1T4KXL4BF' # your Foursquare ID
CLIENT_SECRET = 'K3SSQWZCEOTJEFWHETNWDLUC13VDON2KWGCVNCP1N0AXNO1V' # your Foursquare Secret
VERSION = '20190209' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AVT01CGGJTLVXTNFWSABFLKPYKCRPHLAKFWBICJ1T4KXL4BF
CLIENT_SECRET:K3SSQWZCEOTJEFWHETNWDLUC13VDON2KWGCVNCP1N0AXNO1V


Getting the name and geographical coordinates of the first neighborhood in North York

In [21]:
north_york.loc[0, 'Neighborhood']

'Parkwoods'

In [22]:
neighborhood_latitude = north_york.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [23]:
print('Extracting the url information pertaining to {}, North York, Toronto, Canada.'.format(north_york.loc[0,'Neighborhood']))

Extracting the url information pertaining to Parkwoods, North York, Toronto, Canada.


In [24]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=AVT01CGGJTLVXTNFWSABFLKPYKCRPHLAKFWBICJ1T4KXL4BF&client_secret=K3SSQWZCEOTJEFWHETNWDLUC13VDON2KWGCVNCP1N0AXNO1V&v=20190209&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c71594cdb04f505a4a81c8b'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Extracting the details (name, categories, geographical coordinates) of nearby venues in Parkwoods

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [28]:
print('{} venues were returned by Foursquare'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare


### Explore Neighborhoods in North York
Creating a general function that will return a list of venues within a 1000m radius of the specified neighborhood.

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function to all neighborhoods in the borough of North York. 
The result returned is a dataframe called north_york_venues, with information that will be used for neighborhoods clustering in North York. 

In [30]:
north_york_venues = getNearbyVenues(names=north_york['Neighborhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )
north_york_venues.head()

Parkwoods
Victoria Village
Lawrence Heights,Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park,Don Mills South
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
CFB Toronto,Downsview East
Silver Hills,York Mills
Downsview West
Maple Leaf Park,North Park,Upwood Park
Humber Summit
Newtonbrook,Willowdale
Downsview Central
Bedford Park,Lawrence Manor East
Emery,Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop


In [31]:
print(north_york_venues.shape)
north_york_venues.head()
print('{} nearby venues around North York, within a 1000m radius.'.format(north_york_venues.shape[0]))

(619, 7)
619 nearby venues around North York, within a 1000m radius.


Presenting the number of venues returned for each neighborhood in North York

In [32]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",26,26,26,26,26,26
Bayview Village,13,13,13,13,13,13
"Bedford Park,Lawrence Manor East",40,40,40,40,40,40
"CFB Toronto,Downsview East",19,19,19,19,19,19
Don Mills North,29,29,29,29,29,29
Downsview Central,4,4,4,4,4,4
Downsview Northwest,29,29,29,29,29,29
Downsview West,9,9,9,9,9,9
"Emery,Humberlea",8,8,8,8,8,8


Presenting the number of unique categories from all the returned venues.

In [33]:
print('There are {} unique categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 155 unique categories.


## 3. Analyze Each Neighborhood

In [34]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Store,Bike Shop,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Food & Drink Shop,Food Court,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Presenting all venues in the neighborhoods of North York (within 500 metres radius),and all unique venue categories, in a dataframe.

* Each row represents a venue, one-hot encoded. 
* Each column represents a venue category.

In [35]:
print('The size of dataframe is {}'.format(north_york_onehot.shape))

The size of dataframe is (619, 156)


Grouping rows(venues) by neighborhood, and taking the average of the frequency of occurrance of each category

In [36]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Store,Bike Shop,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Food & Drink Shop,Food Court,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000,0.000000,0.038462,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000,0.038462,0.038462,0.000000,0.00,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.038462,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000,0.000000,0.153846,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,

##### Presenting the Top 5 most common venues for each neighborhood in North York

In [37]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                venue  freq
0         Coffee Shop  0.08
1         Pizza Place  0.08
2         Men's Store  0.04
3            Ski Area  0.04
4  Frozen Yogurt Shop  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.15
1                 Bank  0.15
2        Grocery Store  0.15
3         Intersection  0.08
4   Chinese Restaurant  0.08


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1           Coffee Shop  0.08
2    Italian Restaurant  0.08
3                  Café  0.02
4              Pharmacy  0.02


----CFB Toronto,Downsview East----
                       venue  freq
0                Coffee Shop  0.16
1         Turkish Restaurant  0.11
2          Electronics Store  0.11
3  Middle Eastern Restaurant  0.05
4             Sandwich Place  0.05


----Don Mills North----
                 venue  freq
0  Japanese Restaurant  0.10
1          Pizza Place  

In [38]:
# A function that sorts the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a dataframe and displays the top 5 venues for each neighborhood

In [39]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Pizza Place,Coffee Shop,Pharmacy,Sushi Restaurant,Men's Store
1,Bayview Village,Grocery Store,Japanese Restaurant,Bank,Fast Food Restaurant,Skate Park
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pet Store,Sandwich Place
3,"CFB Toronto,Downsview East",Coffee Shop,Electronics Store,Turkish Restaurant,Soccer Field,Park
4,Don Mills North,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Supermarket
5,Downsview Central,Vietnamese Restaurant,Restaurant,Baseball Field,Yoga Studio,Empanada Restaurant
6,Downsview Northwest,Hotel,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy
7,Downsview West,Spa,Coffee Shop,Shopping Mall,Park,Grocery Store
8,"Emery,Humberlea",Park,Golf Course,Convenience Store,Discount Store,Business Service
9,"Fairview,Henry Farm,Oriole",Clothing Store,Coffee Shop,Japanese Restaurant,Bakery,Sandwich Place


## 4. Clustering Neighborhoods
Using K-Means to cluster the neighborhoods into 4 clusters

In [40]:
# set number of clusters
kclusters = 3

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for the first 20 rows in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])

Creating a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Name', kmeans.labels_)

north_york_merged = north_york

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Pharmacy,Convenience Store,Shopping Mall,Bus Stop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Coffee Shop,Lounge,Portuguese Restaurant,Sporting Goods Shop,Café
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Fast Food Restaurant,Coffee Shop,Vietnamese Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,1,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Supermarket
4,M6B,North York,Glencairn,43.709577,-79.445073,1,Grocery Store,Fast Food Restaurant,Gym,Coffee Shop,Pub


#### Visualizing the Clustering

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighborhood'], north_york_merged['Cluster Name']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if np.isnan(cluster):
        cluster = np.nan_to_num(cluster)
        
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [43]:
north_york_merged.loc[north_york_merged['Cluster Name'] == 0, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,North York,0,Vietnamese Restaurant,Restaurant,Baseball Field,Yoga Studio,Empanada Restaurant


Cluster 1 seems to be characterized by neighborhoods with most common venues comprising 
* restaurants 
* sports (baseball field) and wellness (yoga studio) facilities.

There is only 1 neighborhood in this cluster.

#### Cluster 2

In [44]:
north_york_merged.loc[north_york_merged['Cluster Name'] == 1, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,1,Park,Pharmacy,Convenience Store,Shopping Mall,Bus Stop
1,North York,1,Coffee Shop,Lounge,Portuguese Restaurant,Sporting Goods Shop,Café
2,North York,1,Furniture / Home Store,Clothing Store,Fast Food Restaurant,Coffee Shop,Vietnamese Restaurant
3,North York,1,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Supermarket
4,North York,1,Grocery Store,Fast Food Restaurant,Gym,Coffee Shop,Pub
5,North York,1,Japanese Restaurant,Restaurant,Coffee Shop,Beer Store,Bus Line
6,North York,1,Pharmacy,Coffee Shop,Park,Convenience Store,Bank
7,North York,1,Pizza Place,Coffee Shop,Pharmacy,Sushi Restaurant,Men's Store
8,North York,1,Clothing Store,Coffee Shop,Japanese Restaurant,Bakery,Sandwich Place
9,North York,1,Pizza Place,Coffee Shop,Furniture / Home Store,Sandwich Place,Restaurant


Cluster 2 seems to be characterized by neighborhoods with most common venues comprising 
* food and beverage outlets (coffeeshop, restaurant, sandwich place), 
* pet stores, 
* home and lifestyle (convenience and grocery stores, pharmacies, electronics and discount stores, shopping mall), and
* service facilities (bank, bus stop, business service)

This cluster comprise the most neighborhoods in North York (90%).

#### Cluster 3

In [45]:
north_york_merged.loc[north_york_merged['Cluster Name'] == 2, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,North York,2,Park,Pool,Yoga Studio,Electronics Store,Fireworks Store


Cluster 3 seems to be characterized by neighborhoods with most common venues comprising
* recreational facilities (park)
* sports and wellness facilities (pool and yoga studio)
* general stores (electronics and fireworks).

There is only one neighborhood in this cluster.